In [292]:
import pandas as pd
import numpy as np
import os
import urllib
import chardet
from validate_docbr import CPF, CNPJ

### Aquisição dos Dados e Criação da DataFrame

In [202]:
# definir a função que vai buscar os dados em um intervalo de tempo no site do CEAPS

def busca_dados(inicial, final):
    inicial = 2000
    final = 2023
    caminho_src_data = f'..\\src\\data\\'
    for ano in np.arange(inicial, final+1,1):
        url = f'https://www.senado.gov.br/transparencia/LAI/verba/despesa_ceaps_{ano}.csv'
        try:
            urllib.request.urlretrieve(url,filename=caminho_src_data+f'despesa_ceaps_{ano}.csv')
            print(f'dados de {ano} foram salvos')
        except:
            print(f'sem dados para {ano}')
            
# os dado em csv não estavam na codificação padrão do pandas.read_csv            
# A função encoding_dados lê o arquivo e verifica o encoding mais provável, retornando o mesmo como string
            
def encoding_dados(ano):
    with open(caminho_src_data + f'despesa_ceaps_{ano}.csv', 'rb') as file:
        result = chardet.detect(file.read())
    return result['encoding']

# a função cria_dataframe recebe o ano inicial e final nos quais ela irá buscar 
# na pasta src/data do projeto e incorporar todos em uma unica dataframe pandas

def cria_dataframe(inicial,final, encoding='ISO-8859-1'):
    caminho_src_data = f'..\\src\\data\\' #pasta na qual está salvo os arquivos csv
    df_aux = pd.DataFrame()
    for ano in np.arange(inicial, final + 1,1):
        try:
            arquivo = f'despesa_ceaps_{ano}.csv'
            df_tmp = pd.read_csv(caminho_src_data + arquivo, encoding=encoding, on_bad_lines='skip', delimiter=';',skiprows=1,)
            df_aux = pd.concat([df_aux,df_tmp],ignore_index=True)
        except:
            None
    return df_aux
    
        

### Limpesa dos Dados

In [270]:
df = cria_dataframe(2010,2025)

In [271]:
# verificando as linhas duplicadas
n_linhasduplicadas = df.loc[df.duplicated()].shape[0]
print(f'Nesses dados há {n_linhasduplicadas} linhas duplicadas')


Nesses dados há 0 linhas duplicadas


In [272]:
# Verificando as variáveis
colunas = df.columns.tolist()
colunas

['ANO',
 'MES',
 'SENADOR',
 'TIPO_DESPESA',
 'CNPJ_CPF',
 'FORNECEDOR',
 'DOCUMENTO',
 'DATA',
 'DETALHAMENTO',
 'VALOR_REEMBOLSADO',
 'COD_DOCUMENTO']

In [273]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299026 entries, 0 to 299025
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ANO                299026 non-null  int64  
 1   MES                299026 non-null  int64  
 2   SENADOR            299026 non-null  object 
 3   TIPO_DESPESA       299026 non-null  object 
 4   CNPJ_CPF           299026 non-null  object 
 5   FORNECEDOR         299026 non-null  object 
 6   DOCUMENTO          278790 non-null  object 
 7   DATA               299026 non-null  object 
 8   DETALHAMENTO       180648 non-null  object 
 9   VALOR_REEMBOLSADO  299026 non-null  object 
 10  COD_DOCUMENTO      299024 non-null  float64
dtypes: float64(1), int64(2), object(8)
memory usage: 25.1+ MB


Verifica-se que há mais de uma entrada de Data, com ANO, MES e DATA, a ultima sendo um obj.

In [274]:
df[20000:20050].head()

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
20000,2011,10,ANA RITA,"Passagens aéreas, aquáticas e terrestres nacio...",02.012.862/0001-60,TAM LINHAS AÉREAS SA,9572435571607,04/10/2011,ASSISTENTE PARLAMENTAR TÁURIO LUCILO TESSAROLO...,"771,66",433571.0
20001,2011,10,ANA RITA,"Passagens aéreas, aquáticas e terrestres nacio...",02.012.862/0001-60,TAM LINHAS AÉREAS SA,9572435644021,06/10/2011,SENADORA ANA RITA BRASÍLIA-VITÓRIA 06/10/2011,"771,66",433570.0
20002,2011,10,ANA RITA,"Passagens aéreas, aquáticas e terrestres nacio...",02.012.862/0001-60,TAM LINHAS AÉREAS SA,9572436387029,20/10/2011,SENADORA ANA RITA BRASÍLIA-VITÓRIA 20/10/2011,"771,66",440374.0
20003,2011,10,ANA RITA,"Passagens aéreas, aquáticas e terrestres nacio...",02.428.624/0001-30,TRIP LINHAS AÉREAS SA,6612100720479,27/10/2011,SENADORA ANA RITA BRASÍLIA-VITÓRIA 27/10/2011,"430,56",440372.0
20004,2011,10,ANA RITA,"Passagens aéreas, aquáticas e terrestres nacio...",07.575.651/0001-59,VRG LINHAS AÉREAS SA,12700165816053,31/10/2011,SENADORA ANA RITA VITÓRIA-BRASÍLIA 31/10/2011,"699,13",440371.0


Observado os dados, além de faltantes, o campo DATA contém a mesma informação que ANO e MES.
Épossível criar uma coluna com o dia e remover a data, ou apenas remover o campo DATA.
Vou optar por remover com campo, posterior mente se houver necessidade de incluir o dia em algum tipo de modelo, isso deve ser revisto.

In [275]:
try:
    df = df.drop(["DATA"],axis=1)
except:
    None
df

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2010,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,CENTRAIS ELÉTRICAS DE RONDÔNIA,045216-3,NaN,"59,08",277219.0
1,2010,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,CENTRAIS ELÉTRICAS DE RONDônia,0454206-1,NaN,"40,47",277218.0
2,2010,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,NaN,NaN,5000,229109.0
3,2010,1,ACIR GURGACZ,Divulgação da atividade parlamentar,09.040.457/0001-02,CENTRAL RONDÔNIA.COM,000003,NaN,1500,268264.0
4,2010,1,ACIR GURGACZ,"Locomoção, hospedagem, alimentação, combustíve...",15.896.509/0001-31,POSTO FORTALEZA,018215,NaN,30,268265.0
...,...,...,...,...,...,...,...,...,...,...
299021,2023,1,ZENAIDE MAIA,"Locomoção, hospedagem, alimentação, combustíve...",35.298.330/0001-90,JVC Comercial Ltda Posto Natal,243526,NaN,"392,91",2192830.0
299022,2023,1,ZENAIDE MAIA,"Locomoção, hospedagem, alimentação, combustíve...",35.298.330/0001-90,JVC Comercial Ltda Posto Natal,246907,NaN,"400,04",2192831.0
299023,2023,1,ZENAIDE MAIA,"Locomoção, hospedagem, alimentação, combustíve...",46.795.057/0001-25,Posto WELL,8552,NaN,"383,18",2192832.0
299024,2023,1,ZENAIDE MAIA,"Passagens aéreas, aquáticas e terrestres nacio...",07.575.651/0001-59,GOL,XOEMNV,"Companhia Aérea: GOL, Localizador: XOEMNV. Pas...","3388,79",2193083.0


In [276]:
# Os dados do campo VALOR_REEMBOLSADO estão no formato string com separador decimal ','
# definir a função de muda o separador de ',' para '.'
def muda_separador(coluna,separador_desejado = ',', separador_substituir = '.'):
        df[coluna] = df[coluna].str.replace(separador_desejado,separador_substituir)

In [283]:
#   1\r\n675.55
# df.VALOR_REEMBOLSADO.loc[df.VALOR_REEMBOLSADO == '1\r\n675.55']
df.loc[df.VALOR_REEMBOLSADO == '1\r\n675.55'] = 675.55

In [285]:
muda_separador('VALOR_REEMBOLSADO')
df.VALOR_REEMBOLSADO = df.VALOR_REEMBOLSADO.astype('float')

In [288]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299026 entries, 0 to 299025
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ANO                299026 non-null  float64
 1   MES                299026 non-null  float64
 2   SENADOR            299026 non-null  object 
 3   TIPO_DESPESA       299026 non-null  object 
 4   CNPJ_CPF           299026 non-null  object 
 5   FORNECEDOR         299026 non-null  object 
 6   DOCUMENTO          278790 non-null  object 
 7   DETALHAMENTO       180648 non-null  object 
 8   VALOR_REEMBOLSADO  299025 non-null  float64
 9   COD_DOCUMENTO      299024 non-null  float64
dtypes: float64(4), object(6)
memory usage: 22.8+ MB


Validar os CPFs e os CNPJs

In [301]:
def validar_cpf(valor_cpf):
    cpf = CPF()
    return cpf.validate(valor_cpf)

In [304]:
df.CNPJ_CPF.apply(validate_docbr.CPF().validate)

TypeError: 'float' object is not iterable

In [303]:
validate_docbr.CPF().validate

True

In [ ]:
df_2009.groupby("SENADOR").count().sort_values("VALOR_REEMBOLSADO", ascending = False)

In [146]:
df_2009.isna().sum()

ANO                      0
MES                      0
SENADOR                  0
TIPO_DESPESA             0
CNPJ_CPF               695
FORNECEDOR             695
DOCUMENTO             2134
DATA                   695
DETALHAMENTO         16480
VALOR_REEMBOLSADO        0
COD_DOCUMENTO            0
dtype: int64

In [147]:
df_2009

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2009,12,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,NaN,11/12/2009,NaN,5000.00,222467
1,2009,12,ACIR GURGACZ,Divulgação da atividade parlamentar,02.831.112/0002-09,INTERCOM INTERMEDIAÇÕES E COMUNICAÇÃO INTEGRAD...,000330,09/12/2009,NaN,12620.00,222466
2,2009,1,ADELMIR SANTANA,"Aluguel de imóveis para escritório político, c...",NaN,NaN,NaN,NaN,NaN,12466.95,2009010738101
3,2009,2,ADELMIR SANTANA,"Aluguel de imóveis para escritório político, c...",NaN,NaN,NaN,NaN,NaN,12339.60,2009020738101
4,2009,2,ADELMIR SANTANA,Divulgação da atividade parlamentar,NaN,NaN,NaN,NaN,NaN,731.00,2009020738105
...,...,...,...,...,...,...,...,...,...,...,...
16475,2009,12,WELLINGTON SALGADO DE OLIVEIRA,"Locomoção, hospedagem, alimentação, combustíve...",17.280.959/0001-20,Posto Santa Lúcia Ltda,598,31/12/2009,NaN,100.00,231208
16476,2009,12,WELLINGTON SALGADO DE OLIVEIRA,"Locomoção, hospedagem, alimentação, combustíve...",17.272.550/0001-62,Posto São José Ltda,102478,07/12/2009,NaN,80.00,231231
16477,2009,12,WELLINGTON SALGADO DE OLIVEIRA,"Locomoção, hospedagem, alimentação, combustíve...",02.229.293/0001-09,Posto Tic Tac Mg 010 Ltda,111032,29/12/2009,NaN,50.00,231209
16478,2009,12,WELLINGTON SALGADO DE OLIVEIRA,"Locomoção, hospedagem, alimentação, combustíve...",06.098.111/0001-69,Tudo de Bom Restaurante Ltda,12456,09/12/2009,NaN,80.00,224305
